# Getting started

This notebook provides an introduction to the tools that will be used for the _hands-on_ sessions of the MOOC "Path Integrals in Atomistic Modeling". Please follow the instructions to make sure that you have all the necessary components in place. 

## Installation

The notebooks associated with this course rely on some basic Python packages. If executing the following cell returns any errors, you should install the corresponding packages

In [ ]:
import numpy as np
import matplotlib.pyplot as plt

You should also have [i-PI](https://ipi-code.org) installed, and accessible from the path. This might require some more effort: if everything is configured correctly the following cell should return a prompt, and a message stating that 

```Simulation has already run for total_steps, will not even start. Modify total_steps or step counter to continue.```

In [ ]:
!i-pi 0-getting_started/do_nothing.xml

... and the following cell should generate a help string explaining the syntax for running `i-pi-driver`

In [ ]:
!i-pi-driver